<a href="https://colab.research.google.com/github/JayVeer18/CUDA-Programs/blob/main/Factorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Step:1

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

### Step:2

In [ ]:

!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

### Step:3

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


### Step-4

In [ ]:
pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

### Step-5

In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


### Factorial using Parallel Reduction 

In [33]:
%%cu
#include <iostream>
#include <math.h>
#include <stdio.h>
using namespace std;

__global__ void factorial(int* input)
{
	const int tid = threadIdx.x;

	auto step_size = 1;
	int number_of_threads = blockDim.x;

	while (number_of_threads > 0)
	{
		if (tid < number_of_threads) // still alive?
		{
			const auto fst = tid * step_size * 2;
			const auto snd = fst + step_size;
			input[fst] *= input[snd];
		}

		step_size <<= 1; 
		number_of_threads >>= 1;
	}
}

int nextPowerOf2(int n) 
{ 
    n--; 
    n |= n >> 1; 
    n |= n >> 2; 
    n |= n >> 4; 
    n |= n >> 8; 
    n |= n >> 16; 
    n++; 
    return n; 
}

int main()
{
	int n =12;
  scanf("%d",&n);
  int count = nextPowerOf2(n);
	const int size = count * sizeof(int);
	int h[count];
  for(int i=0;i<n;i++){
      h[i] = i+1;
  }
 for(int i=n;i<count;i++){
      h[i] = 1;
  }

	int* d; 
	cudaMalloc(&d, size);
	cudaMemcpy(d, h, size, cudaMemcpyHostToDevice);

	factorial <<<1, count / 2 >>>(d);

	int result;
	cudaMemcpy(&result, d, sizeof(int), cudaMemcpyDeviceToHost);

	cout << "factorial is " << result << endl;

	cudaFree(d);
	return 0;
}

factorial is 479001600

